In [1]:
import scipy.stats as spt
import numpy as np
import pandas as pd
import seaborn as sb
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.weightstats as sw
import matplotlib.pyplot as plt



In [2]:
# Two sample t test

fres = np.array([3.7,4.3,2.5,3.3,3.6,3.1])
soph = np.array([1.8,4.2,4.1,2.2,3.2,3.8])
twosam = spt.ttest_ind(fres,soph) # ind는 independent한 집단. 모분산이 다른경우에는 무언가를 False로.
print(twosam)

print('t statistic = %.3f, p-value = %.3f'%(twosam))


print('1학년 :', np.mean(fres))
print('2학년 :', np.mean(soph))

Ttest_indResult(statistic=0.41473113113479565, pvalue=0.6870956353065248)
t statistic = 0.415, p-value = 0.687
1학년 : 3.4166666666666674
2학년 : 3.216666666666667


모분산이 같을것이냐 다를것이냐 

피밸류가 귀무가설 유의수준에서 크냐 작냐.  알파보다 크냐 작냐만 기억을 해주세요

In [3]:
# Dataframe

data1=pd.DataFrame(data=np.array([[3.7,1.8,3.3,4.1],[4.3,4.2,3.7,3.8],[2.5,4.1,3.4,3.5],[3.3,2.2,3.9,3.2],[3.6,3.2,np.nan,2.3]]),
                   columns=['fre','sop','jun','sen'])
print(data1['jun'],'\n')

sop = data1['sop']
jun = data1['jun']
jun = jun.fillna(jun.mean()) # 결측치에 해당하는 부분을 어떻게 처리할 것인가에 대한 부분. 결측치에 평균을 넣은 상태이다.

print(jun,'\n')

tsam = spt.ttest_ind(sop,jun)
print(tsam)

0    3.3
1    3.7
2    3.4
3    3.9
4    NaN
Name: jun, dtype: float64 

0    3.300
1    3.700
2    3.400
3    3.900
4    3.575
Name: jun, dtype: float64 

Ttest_indResult(statistic=-0.9550271234200176, pvalue=0.36752757550294646)


이건 판다스로 해보는거고.

아래는 쌍체 검정 모집단이 대응이 되는 경우에 해당하며 실제로 많이 쓰인다. 약을 복용한 후, 코칭이나 훈련 등의 전후 비교. 모집단이 같은 사람으로서 대응이 될 것이다. 이때

모분산이 다른건 다른 집단, 같은 집단일 경우는 쌍체검정

In [4]:
# Paired t test

before=np.array([68,61,60,68,67,64,66,67,66,67,72,74,61,71,58,77])
after=np.array([56,55,67,62,59,67,50,60,59,53,60,65,62,61,64,57])

pairsam = spt.ttest_rel(before,after) # rel의 약자는 모르지만. 전과 후에 대한 t검정을 할 수 있고 그때 p_value 값을 통해서, 조치나 약에 효능이 있었나 없었나를 볼 수 있다.
print(pairsam)

Ttest_relResult(statistic=3.562626515972126, pvalue=0.002834843356363954)


효과에 대한 검증을 알고 싶으면 이것을 사용해서 간단하게 통계적으로는 pvalue가 얼마보다 작게 나와서 유의미했다고 증명을 해줄수 있다.

statsmodels는 import 해서 쓸 때 찾기 어려우니 헤메지 마시고 여기...쓰세요.

In [5]:
# Statsmodels를 이용한 t test

fre=data1['fre']
sen=data1['sen']


# 가설에는 two-sided / larger / smaller 3가지, 분산에는 pooled, unequal 2가지, value는 두 집단의 평균 차이가 얼마인지 나타내는 수
tsams=sw.ttest_ind(fre,sen,alternative='two-sided',usevar='pooled',value=0)  
print('tstat = \t',tsams[0],'\np-val = \t',tsams[1],'\ndegree of freedom = \t',tsams[2])



# 쌍체검정 (One-sided) 을 여기서도 할 수 있어요!

ptsams = sw.ttost_paired(before,after,0.1,0.2)
print(ptsams)

tstat = 	 0.23453251491009858 
p-val = 	 0.820462752846746 
degree of freedom = 	 8.0
(0.9982467120464241, (3.510806493921623, 0.0015764190017186505, 15.0), (3.458986471871119, 0.9982467120464241, 15.0))


자유도는 여기서 준건데 p에 대한 파라미터니까 크게 뭐 중요한건 아니고.

p-val로 검정을 할 수 있다는 거가 중요한거고.

대부분 사이파이를 쓰지만 스탯츠모델스가 기능이 좋은게 많아서 익숙해지면 좋겠습니다. 

이원분석.

Moore 라는 내장 자동차 데이터. 이 데이터를 띄워보면 

In [6]:
# Two-way ANOVA from statsmodels // Referred : https://www.statsmodels.org/stable/anova.html#module-statsmodels.stats.anova

moore = sm.datasets.get_rdataset("Moore", "carData", cache=True)

data2 = moore.data

data2 = data2.rename(columns={"partner.status":"partner_status"}) # rename

moore_lm = ols('conformity ~ C(fcategory, Sum)*C(partner_status, Sum)',data=data2).fit()  # 회귀분석이 약간 들어가 있다.
# 물결이 독립 팩터들을 통해 분석을 하겠다는 것. 
# conformity = y
# fcategory = class. level 같은 것.
# partner_status 실제 데이터셋에 있는...
# 덧셈으로 하게 되면 교호작용이 없이 테스트 하겠다.
# 곱셈은 교호작용을 고려하고 테스트를 하겠다는 것.
# 덧셈 곱셈을 다 쓰기도 한다고 하셨다.

table = sm.stats.anova_lm(moore_lm, typ=2) # ANOVA 모델 만드는 것. 
# typ은 보통 2로 쓰는데, interaction effect를 처리하는 방법을 고르는 것인데 건들일은 거의 없다. 
# 이상치를 처리하거나, 선형모델같은 것을 예로 든다면 분산분석도 regalazation 이라고 람다에다가 어쩌고 하는 걸 더한다고 한다. 이게 typ에서 처리를 할 것이다. 
# 분산분석을 할 때, y=c1x1+c2x2+c3x1x2 여기 앞에 c1, c2, c3같은 계수가 0이 될 때를 생각하는 것.

print(table)

                                              sum_sq    df          F  \
C(fcategory, Sum)                          11.614700   2.0   0.276958   
C(partner_status, Sum)                    212.213778   1.0  10.120692   
C(fcategory, Sum):C(partner_status, Sum)  175.488928   2.0   4.184623   
Residual                                  817.763961  39.0        NaN   

                                            PR(>F)  
C(fcategory, Sum)                         0.759564  
C(partner_status, Sum)                    0.002874  
C(fcategory, Sum):C(partner_status, Sum)  0.022572  
Residual                                       NaN  


요인별 합으로 구분하기 때문에, 수식으로 쓰게되면 일종의 y=c1x1+c2x2+c3x1x2 가 되기 때문에 listsquare 인 ols를 쓴다고 한다.


sum_sq 편차들의 제곱합

우리가 봐야하는 것은 PR(p-val).0.75니까 영향이 거의 없고. 0.0028은 굉장히 영향이 있네요.

교호작용 0.02은 영향이 있네요.

그래서 어떤 레벨에서 문제가 생기는지 나누어서 테스트를 더 할 수도 있을 것입니다.

밑에서는 사후검정 더.

In [ ]:
# Excel file read
df = pd.read_csv('score1.csv')
print(df)

print(df.grade.unique())

model = ols('score ~ C(grade)', df).fit()
anova_lm(model)

ols 해서 볼 것은 결국

아노바 테이블이 간단하게 나오는데 우리가 볼 것은 PR(p-val). 보고 귀무가설을 기각할지 안할지 결정하고.

In [ ]:
# One-way에서 scipy.stats.f_oneway() 를 사용할 때의 결측치 제거

# data[~np.isnan(data)]

qq 플롯등을 이용해서 요인이 영향이 있는지 없는지 검정 방법들이 있지만, 

오차의 가정점검: 분산분석 뿐만 아니라 회귀분석 등을 사용할때도 오차에 가정이 들어가기 때문에 많이 쓴다.

levene, bartlett 둘 중 하나써도 상관은 없다. shapiro도 보수적이긴 하다.

보통 더 보수적인것을 쓰는 것을 추천한다.

factor 안의 레벨 간 검정을 해도 되고 전체 데이터 셋에서 검정을 해주어도 되지만, 레벨 간별로 큰 문제가 없다면 shapiro 테스트는 전체 테스트를 해주어도 좋다.

레벨간의 interaction이 없다면 레벨별로 해도 되지만 여기 나온 것처럼 grade 전체를 해도 된다.

레벨 안에서는 등분산이 의미가 거의 없다. 거기서 샘플링 여러번 해서 레벨 안에서도 한 단계 나눈다고 할 수도 있겠지만 그 안에선 거의 필요없고. (resicualsvs fitted 그래프에서 줄별로 테스트를 한다고 생각하면 된다)

펙터 레벨별로 나누어줘도 큰 상관이 없다. 

In [ ]:
# 오차의 가정 점검

print(spt.shapiro(df.score[df.grade=='sen']),'\n') # 정규성 테스트

print(spt.levene(df.score[df.grade=='fre'],df.score[df.grade=='sop'],df.score[df.grade=='jun'],df.score[df.grade=='sen']),'\n') # 등분산성-여유있음

print(spt.bartlett(df.score[df.grade=='fre'],df.score[df.grade=='sop'],df.score[df.grade=='jun'],df.score[df.grade=='sen'])) # 등분산성-더 보수적

오차 점검이 잘 되었다고 합시다. p-val이 매우 커서 귀무가설을 깨기 때문에 사후검정으로 넘어가보자.

여기 사후 검정은 레벨 안의 검정이다. 

In [ ]:
# 사후 검정

from statsmodels.sandbox.stats.multicomp import MultiComparison
from statsmodels.stats.multicomp import pairwise_tukeyhsd

comp = MultiComparison(df.score, df.grade) # 안에서 레벨이 구분이 되어있는데, 어떤 애들이 구분이 되어있는지 테스트 하는 것 MultiComparison
# 차이가 난다는 기준 즉 기각역을 셋팅하는 과정이 매우 중요한데, 
# 


# Bonferroni 본펠로니 - 제일 보수적
opt = comp.allpairtest(spt.ttest_ind, method='bonf') # 전체 레벨 개수로 유의수준을 쪼개서, 5개면 결국 유의수준은 1%가 되어버린다. 요인간의 차이를 인정하지 않을 가능성이 커짐.
print(opt[0])

# Tukey's HSD 튜키 - 제일 많이 씀
hsd = pairwise_tukeyhsd(df['score'], df['grade'], alpha=0.05) # 너무 tight 하거나 loose 하지 않은 중간정도. 공학적으로 많이 쓴다.
hsd.summary()



섀키 , 피셔 등도 있지만 위의 둘 을 제일 많이 쓴다. duncen ficher등

0.008도 절사된 것일텐데. 귀무가설을 기각할 수 없는 상황이다. 그룹간 사후비교를 했을때, 차이가 별ㄹ 나타나지 않는다고 bonferroni는 차이를 인정하지않는다고 했다.

tukey에선 되는데 bonferroni에서는 안된다고 하는 경우도 허다하기 때문에

분석을 알고 있는 사람이 보게 되면 이런 디테일을 많이본다. 어떤 분석은 더 보수적으로 해야하지 않느냐, 혹은 러프한 분석방법을 써도 된다고 이야기를 한다고.

결국 사람이 정함..

기각이 되었을 때 어떤 부분에서 문제가 있느냐를 찾는 과정이기 때문에 중요하며,

명령어 한 줄로 찾을 수 있기 때문에 (statsmodels.sandbox.stats.multicomp)



데이터셋을 보니, 아까는 그레이드만 나왔는데 클래스가 하나더 나왔다.

이때 length를 찾아보면 data length가 같은 것 끼리 되어있음을 확인 할 수 있다.

만약 데이터 렝스가 다르면 다르다고 옵션을 넣어주어야 하는 것으로 알고 있다.

밑에 + 로 해둔 것은 덴시티플롯이나 박스플롯을 봤더니 요인끼리 큰 영향을 끼치지 않아서 +를 쓰셨다고.

In [ ]:
# Two-way ANOVA

data2 = pd.read_csv('score2.csv')
data2.head()

In [ ]:
#data2.groupby('grade').agg(len)
data2.groupby('class').agg(len)
#data2.groupby(['grade', 'class']).agg(len)

In [ ]:
model2 = ols('score ~ C(grade)*C(class)', data=data2).fit() # 교호작용이 없을 것으로 생각하고 +를 썼다. 
anova_lm(model2)

p-val 을 보았을 때, 영향이 있다고 볼 수 있다.

사람이 체크해야 하는건 항상 오차 가정점검을 꼭 해야한다는 것. 레지듀얼이나 큐큐 플롯 등.

정규성에도 레지듀얼이나 큐큐 플롯등 한 가지 증명할 때도 여러 테스트를 같이 넣어서 검정을 해야 신뢰를 받을 수 있다.

너무 한 가지 방법을 사용했을 때 문제가 생길 수 있기 때문에, 등분산성이든 체크를 하고 검정하는 툴을, 여러가지 방법을 사용해야 합니다.

In [ ]:
# statsmodels anova_lm type : Referred https://jooskorstanje.com/anova-types-of-sums-of-squares-notebook.html

weekday = ['sat', 'sat', 'sat', 'sat', 'sat', 'sat', 'sun', 'sun', 'sun', 'sun']
weather = ['rain', 'rain', 'rain', 'rain', 'rain', 'sun', 'sun', 'sun', 'sun', 'sun']
sales = [100, 100, 100, 100, 100, 10000, 10000, 10000, 10000, 10000]

data = pd.DataFrame({'weekday': weekday, 'weather': weather, 'sales': sales})
data

아래도 ANOVA 분석 이야기입니다~ csv넣어서 아래도 직접 해보기.

In [ ]:
# Type I tells us that weekday is more important. The interaction effect is not signifcant.
lm = ols('sales ~ C(weekday)*C(weather)',data=data).fit()
table = sm.stats.anova_lm(lm, typ=1) # Type 1 ANOVA DataFrame
print(table)

In [ ]:
# Type II tells us that weather is more important. There is no interaction effect.
lm = ols('sales ~ C(weekday) + C(weather)',data=data).fit()
table = sm.stats.anova_lm(lm, typ=2) # Type 2 ANOVA DataFrame
print(table)

type 2를 가장 많이 쓰고. type3의 경우 

분산분석은 카테고리얼. 회귀분석은 수치적인것.

분산분석은 조금더 범용적이기 때문에 주로 쓴다. 무엇으로해도 상관은 없지만 회귀분석은 함수를 만드는 것이기 때문에, 카테고리 값이 들어가면 완전히 무너진다.

In [ ]:
# Type III tells us that weekday is more important. The interaction effect is not signifcant.
lm = ols('sales ~ C(weekday)*C(weather)',data=data).fit()
table = sm.stats.anova_lm(lm, typ=3) # Type 3 ANOVA DataFrame
print(table)